In [1]:
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [19]:
from sklearn import preprocessing

In [37]:
from sklearn.linear_model import LogisticRegression

In [34]:
from sklearn.metrics import classification_report

In [11]:
train_data = pd.read_csv('train.csv')

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [13]:
train_data = train_data.fillna({'Age': train_data.Age.median()})

In [14]:
train_data['Age_levels'] = pd.cut(train_data['Age'],bins=[0,12,18,79],labels=['Age_1','Age_2','Age_3'], right=False)

In [15]:
train_data.head(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Age_levels
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,Age_3
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,Age_3
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,Age_3


In [16]:
df_train = pd.get_dummies(train_data, columns=['HomePlanet','CryoSleep','Destination','VIP', 'Age_levels'])

In [17]:
df_train.drop(['Name','Cabin','PassengerId', 'Age'], axis = 1, inplace = True)

In [53]:
df_train

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Age_levels_Age_1,Age_levels_Age_2,Age_levels_Age_3,Transported
0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,0,0,1,1,0,0,0,1,0
1,109.0,9.0,25.0,549.0,44.0,1,0,0,1,0,0,0,1,1,0,0,0,1,1
2,43.0,3576.0,0.0,6715.0,49.0,0,1,0,1,0,0,0,1,0,1,0,0,1,0
3,0.0,1283.0,371.0,3329.0,193.0,0,1,0,1,0,0,0,1,1,0,0,0,1,0
4,303.0,70.0,151.0,565.0,2.0,1,0,0,1,0,0,0,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,6819.0,0.0,1643.0,74.0,0,1,0,1,0,1,0,0,0,1,0,0,1,0
8689,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,0,1,0,0,0,1,0
8690,0.0,0.0,1872.0,1.0,0.0,1,0,0,1,0,0,0,1,1,0,0,0,1,1
8691,0.0,1049.0,0.0,353.0,3235.0,0,1,0,1,0,1,0,0,1,0,0,0,1,0


In [21]:
label_encoder = preprocessing.LabelEncoder()

In [22]:
df_train['Transported']= label_encoder.fit_transform(df_train['Transported'])

In [54]:
df_train

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Age_levels_Age_1,Age_levels_Age_2,Age_levels_Age_3,Transported
0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,0,0,1,1,0,0,0,1,0
1,109.0,9.0,25.0,549.0,44.0,1,0,0,1,0,0,0,1,1,0,0,0,1,1
2,43.0,3576.0,0.0,6715.0,49.0,0,1,0,1,0,0,0,1,0,1,0,0,1,0
3,0.0,1283.0,371.0,3329.0,193.0,0,1,0,1,0,0,0,1,1,0,0,0,1,0
4,303.0,70.0,151.0,565.0,2.0,1,0,0,1,0,0,0,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,6819.0,0.0,1643.0,74.0,0,1,0,1,0,1,0,0,0,1,0,0,1,0
8689,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,0,1,0,0,0,1,0
8690,0.0,0.0,1872.0,1.0,0.0,1,0,0,1,0,0,0,1,1,0,0,0,1,1
8691,0.0,1049.0,0.0,353.0,3235.0,0,1,0,1,0,1,0,0,1,0,0,0,1,0


In [24]:
df_train.fillna(df_train[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].median(),
                inplace=True) 

In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   RoomService                8693 non-null   float64
 1   FoodCourt                  8693 non-null   float64
 2   ShoppingMall               8693 non-null   float64
 3   Spa                        8693 non-null   float64
 4   VRDeck                     8693 non-null   float64
 5   Transported                8693 non-null   int64  
 6   HomePlanet_Earth           8693 non-null   uint8  
 7   HomePlanet_Europa          8693 non-null   uint8  
 8   HomePlanet_Mars            8693 non-null   uint8  
 9   CryoSleep_False            8693 non-null   uint8  
 10  CryoSleep_True             8693 non-null   uint8  
 11  Destination_55 Cancri e    8693 non-null   uint8  
 12  Destination_PSO J318.5-22  8693 non-null   uint8  
 13  Destination_TRAPPIST-1e    8693 non-null   uint8

In [ ]:
target_trans = df_train.pop('Transported')

In [31]:
# insert column using insert(position,column_name,
# first_column) function
df_train.insert(18, 'Transported', target_trans)

In [32]:
df_train

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Age_levels_Age_1,Age_levels_Age_2,Age_levels_Age_3,Transported
0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,0,0,1,1,0,0,0,1,0
1,109.0,9.0,25.0,549.0,44.0,1,0,0,1,0,0,0,1,1,0,0,0,1,1
2,43.0,3576.0,0.0,6715.0,49.0,0,1,0,1,0,0,0,1,0,1,0,0,1,0
3,0.0,1283.0,371.0,3329.0,193.0,0,1,0,1,0,0,0,1,1,0,0,0,1,0
4,303.0,70.0,151.0,565.0,2.0,1,0,0,1,0,0,0,1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,6819.0,0.0,1643.0,74.0,0,1,0,1,0,1,0,0,0,1,0,0,1,0
8689,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,0,1,0,0,0,1,0
8690,0.0,0.0,1872.0,1.0,0.0,1,0,0,1,0,0,0,1,1,0,0,0,1,1
8691,0.0,1049.0,0.0,353.0,3235.0,0,1,0,1,0,1,0,0,1,0,0,0,1,0


In [33]:
X = df_train.iloc[:,:-1]
y = df_train.iloc[:,-1]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42)

In [40]:
model_lr = LogisticRegression(max_iter=100)

In [41]:
model_lr.fit(X_train, y_train)

C:\Users\comar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [43]:
y_pred = model_lr.predict(X_test)

In [44]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1301  413]
 [ 365 1399]]
              precision    recall  f1-score   support

           0       0.78      0.76      0.77      1714
           1       0.77      0.79      0.78      1764

    accuracy                           0.78      3478
   macro avg       0.78      0.78      0.78      3478
weighted avg       0.78      0.78      0.78      3478



In [45]:
test_data = pd.read_csv('test.csv')

In [61]:
test_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Age_levels
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,Age_3
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,Age_3
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,Age_3
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,Age_3
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,Age_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,Age_3
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,Age_3
4274,9271_01,Mars,True,D/296/P,55 Cancri e,26.0,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,Age_3
4275,9273_01,Europa,False,D/297/P,NaN,26.0,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,Age_3


In [46]:
test_data = test_data.fillna({'Age': test_data.Age.median()})

In [47]:
test_data['Age_levels'] = pd.cut(test_data['Age'],bins=[0,12,18,79],labels=['Age_1','Age_2','Age_3'], right=False)

In [49]:
df_test = pd.get_dummies(test_data, columns=['HomePlanet','CryoSleep','Destination','VIP', 'Age_levels'])

In [50]:
df_test.drop(['Name','Cabin','PassengerId', 'Age'], axis = 1, inplace = True)

In [51]:
df_test

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Age_levels_Age_1,Age_levels_Age_2,Age_levels_Age_3
0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,1,1,0,0,0,1
1,0.0,9.0,0.0,2823.0,0.0,1,0,0,1,0,0,0,1,1,0,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,1,0,0,1,0,0,0,1
3,0.0,6652.0,0.0,181.0,585.0,0,1,0,1,0,0,0,1,1,0,0,0,1
4,10.0,0.0,635.0,0.0,0.0,1,0,0,1,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,1,1,0,0,0,1
4273,0.0,847.0,17.0,10.0,144.0,1,0,0,1,0,0,0,1,1,0,0,0,1
4274,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,1,0,0,1,0,0,0,1
4275,0.0,2680.0,0.0,0.0,523.0,0,1,0,1,0,0,0,0,1,0,0,0,1


In [55]:
df_test.fillna(df_test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].median(),
                inplace=True)    

In [56]:
X_test=df_test.iloc[:,:]
y_pred = model_lr.predict(X_test)

In [57]:
test_pred = pd.concat([df_test, pd.DataFrame(y_pred)], axis=1)

In [58]:
test_pred

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Age_levels_Age_1,Age_levels_Age_2,Age_levels_Age_3,0
0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,1,1,0,0,0,1,1
1,0.0,9.0,0.0,2823.0,0.0,1,0,0,1,0,0,0,1,1,0,0,0,1,0
2,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,1,0,0,1,0,0,0,1,1
3,0.0,6652.0,0.0,181.0,585.0,0,1,0,1,0,0,0,1,1,0,0,0,1,1
4,10.0,0.0,635.0,0.0,0.0,1,0,0,1,0,0,0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,1,1,0,0,0,1,1
4273,0.0,847.0,17.0,10.0,144.0,1,0,0,1,0,0,0,1,1,0,0,0,1,0
4274,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,1,0,0,1,0,0,0,1,1
4275,0.0,2680.0,0.0,0.0,523.0,0,1,0,1,0,0,0,0,1,0,0,0,1,1


In [59]:
test_pred.rename(columns={0: 'Transported'},
          inplace=True, errors='raise')

In [60]:
test_pred

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Age_levels_Age_1,Age_levels_Age_2,Age_levels_Age_3,Transported
0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,1,1,0,0,0,1,1
1,0.0,9.0,0.0,2823.0,0.0,1,0,0,1,0,0,0,1,1,0,0,0,1,0
2,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,1,0,0,1,0,0,0,1,1
3,0.0,6652.0,0.0,181.0,585.0,0,1,0,1,0,0,0,1,1,0,0,0,1,1
4,10.0,0.0,635.0,0.0,0.0,1,0,0,1,0,0,0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,1,1,0,0,0,1,1
4273,0.0,847.0,17.0,10.0,144.0,1,0,0,1,0,0,0,1,1,0,0,0,1,0
4274,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,1,0,0,1,0,0,0,1,1
4275,0.0,2680.0,0.0,0.0,523.0,0,1,0,1,0,0,0,0,1,0,0,0,1,1


In [63]:
output = pd.DataFrame({'PassengerId':test_data.PassengerId, 'Transported':test_pred.Transported})
output.to_csv('my_submission.csv', index=False)

In [65]:
output["Transported"] = output["Transported"].astype(bool)

In [66]:
output

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [67]:
output.to_csv('my_submission.csv', index=False)

In [68]:
my = pd.read_csv('my_submission.csv')

In [69]:
my

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
